<a href="https://colab.research.google.com/github/acordeir/DSExpoDemo/blob/main/DSExpoDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers
!pip install huggingface_hub
!pip install langchain
!pip install bitsandbytes accelerate xformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [7]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [18]:
# Use a pipeline for later
from transformers import pipeline

'''
summarize_pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )
'''

generation_pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [19]:
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')

In [10]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


In [20]:
#summarize_llm = HuggingFacePipeline(pipeline = summarize_pipe, model_kwargs = {'temperature':0.1})
generation_llm = HuggingFacePipeline(pipeline = generation_pipe, model_kwargs = {'temperature':0.1})

In [26]:
system_prompt1 = "You are an advanced assistant that excels at summarization. "
instruction1 = "Summarize the following text, be sure to include all important information:\n\n {text}"
template1 = get_prompt(instruction1, system_prompt1)
print(template1)

system_prompt2 = "You are an advanced assistant that excels at text generation"
instruction2 = "Generate about 50 words based on the following prompt:\n\n {text}"
template2 = get_prompt(instruction2, system_prompt2)
print(template2)

#summarize_prompt = PromptTemplate(template=template, input_variables=["text"])
#summarize_llm_chain = LLMChain(prompt=summarize_prompt, llm=summarize_llm)

[INST]<<SYS>>
You are an advanced assistant that excels at summarization. 
<</SYS>>

Summarize the following text, be sure to include all important information:

 {text}[/INST]
[INST]<<SYS>>
You are an advanced assistant that excels at text generation
<</SYS>>

Generate about 50 words based on the following prompt:

 {text}[/INST]


In [27]:
summarize_prompt = PromptTemplate(template=template1, input_variables=["text"])
summarize_llm_chain = LLMChain(prompt=summarize_prompt, llm=generation_llm)

generation_prompt = PromptTemplate(template=template2, input_variables=["text"])
generation_llm_chain = LLMChain(prompt=generation_prompt, llm=generation_llm)

In [23]:
story = "Once upon a time in the bustling city of New York, there lived a diligent banker named Benjamin. He had spent years working at the prestigious Sterling Bank, where precision and attention to detail were paramount. Benjamin was renowned for his unwavering commitment to accuracy, but one fateful day, the pressures of his job took a toll on him. It was a Friday afternoon, and the bank was abuzz with clients eager to secure their weekend funds. Benjamin was tasked with a critical wire transfer for a prominent client. As he hastily input the numbers, his mind wandered to a looming deadline and personal issues. In that moment of distraction, he entered the wrong account number.The mistake was discovered when the client called to inquire about the missing funds. Panic coursed through Benjamin as he realized the gravity of his error. His once-impeccable reputation now hung in the balance, and the bank had to rectify the situation quickly, reimbursing the client's lost funds.The bank's senior management, although sympathetic to Benjamin's previous record, had no choice but to terminate his employment. His termination served as a stark reminder that even the most dedicated individuals could falter under the weight of stress and distractions, ultimately leading to the loss of their livelihoods. Benjamin learned a harsh lesson that day, forever haunted by the one costly mistake that changed the course of his career."

In [24]:
#text = "how are you today?"
output = summarize_llm_chain.run(story)

parse_text(output)

  Benjamin, a diligent banker at Sterling Bank in New York, made a costly mistake while working on a
critical wire transfer for a prominent client. Despite his reputation for accuracy, he accidentally
entered the wrong account number, resulting in the client losing their funds. When the client called
to inquire about the missing money, Benjamin realized the gravity of his error and panicked. The
bank's senior management had no choice but to terminate his employment due to the mistake, despite
his previous record of excellence. The incident served as a harsh reminder that even the most
dedicated individuals can make mistakes under pressure, leading to serious consequences for their
career. Benjamin was left to grapple with the consequences of his error, forever changed by the
experience.




## Text Cleaner

In [28]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install scrubadub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.5/636.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 84.1 MB/s eta 0:00:00
  Attempting uninstall: textblob
    Found existing installation: textblob 0.17.1
    Uninstalling textblob-0.17.1:
      Successfully uninstalled textblob-0.17.1


In [29]:
import scrubadub
import nltk
nltk.download('punkt')

def privatize_text(text):
  scrubber = scrubadub.Scrubber()
  scrubber.add_detector(scrubadub.detectors.TextBlobNameDetector)
  privatized_text = scrubber.clean(text)
  return privatized_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [30]:
parse_text(privatize_text(output))


  {{NAME}}, a diligent banker at {{NAME}} {{NAME}} in {{NAME}} {{NAME}}, made a costly mistake while
working on a critical wire transfer for a prominent client. Despite his reputation for accuracy, he
accidentally entered the wrong account number, resulting in the client losing their funds. When the
client called to inquire about the missing money, {{NAME}} realized the gravity of his error and
panicked. The bank's senior management had no choice but to terminate his employment due to the
mistake, despite his previous record of excellence. The incident served as a harsh reminder that
even the most dedicated individuals can make mistakes under pressure, leading to serious
consequences for their career. {{NAME}} was left to grapple with the consequences of his error,
forever changed by the experience.




## Image Privatize

In [31]:
!pip install opencv-python

In [32]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math

def privatize_img(image):
  #Used as reference: https://huggingface.co/spaces/sciling/Face_and_Plate_License_Blur/tree/main/models

  plate_blur = 100
  face_blur = 100

  #model_face = torch.load('face_model.pt')
  #model_face.eval()
  '''
  model_plate = yolov5.load('keremberke/yolov5m-license-plate', device="cpu")
  # # set model parameters
  model_plate.conf = 0.25  # NMS confidence threshold
  model_plate.iou = 0.45  # NMS IoU threshold
  model_plate.agnostic = False  # NMS class-agnostic
  model_plate.multi_label = False  # NMS multiple labels per box
  model_plate.max_det = 1000  # maximum number of detections per image

  results_plate = model_plate(image, augment=True)
  boxes_plate_list = results_plate.pred[0][:, :4].cpu().numpy().tolist()
  for box in boxes_plate_list:
      ROI = image[int(box[1]):int(box[3]), int(box[0]):int(box[2])]

      blur_value = (int(plate_blur) * 2 - 1)
      blur = cv2.GaussianBlur(ROI, (blur_value, blur_value), 20, cv2.BORDER_DEFAULT)
      # Insert ROI back into image
      image[int(box[1]):int(box[3]), int(box[0]):int(box[2])] = blur

  # Converting BGR image into a RGB image
  #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  '''
  image = np.array(image)

  face_detect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
  face_data = face_detect.detectMultiScale(image, 1.3, 5)

  # Draw rectangle around the faces which is our region of interest (ROI)
  for (x, y, w, h) in face_data:
      p1 = (int(x), int(y))
      p2 = (int(x+w), int(y+h))

      circle_center = ((p1[0] + p2[0]) // 2, (p1[1] + p2[1]) // 2)
      circle_radius = int(math.sqrt(w * w + h * h) // 2)

      ROI = np.zeros(image.shape, dtype='uint8')
      cv2.circle(ROI, circle_center, circle_radius, (255, 255, 255), -1)
      # applying a gaussian blur over this new rectangle area
      blur_value = (int(face_blur) * 2 - 1)
      blur = cv2.GaussianBlur(image, (blur_value, blur_value), 20, cv2.BORDER_DEFAULT)

      image = np.where(ROI > 0, blur, image)

  return image

def plotImages(img):
    plt.imshow(img)
    plt.axis('off')
    plt.style.use('seaborn')
    plt.show()

## PDF Reader

In [33]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 3.5 MB/s eta 0:00:00


In [5]:
from pypdf import PdfReader

reader = PdfReader("disclosure_193009_2020-12-30.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [110]:
from pypdf import PdfReader


def preview_text(file):
  reader = PdfReader(file)
  page = reader.pages[0]
  text = page.extract_text()
  wrapped_text = textwrap.shorten(text, width=300)
  return str(wrapped_text)


In [113]:
def privatize_pdf_summary(filepath):
  reader = PdfReader(filepath)
  number_of_pages = len(reader.pages)
  summaries = []
  for p in reader.pages:
    text = p.extract_text()
    summary = privatize_text(generation_llm_chain.run(text))
    summaries.append(summary)
  return '\n'.join(summaries)



In [89]:
privatize_pdf_summary('APS_Demo_DSExpo.pdf')

"  Sure, here is a summary of the letter:\nMr. {{NAME}} {{NAME}}, a 58-year-old male patient, was admitted to {{NAME}} {{NAME}} {{NAME}} {{NAME}} on {{NAME}} 18th with a history of coronary artery disease (CAD) and congestive heart failure (CHF). During his hospitalization, he experienced worsening chest pain, shortness of breath, and palpitations. {{NAME}} diagnostic evaluations confirmed severe coronary artery disease with impaired left ventricular function. The {{NAME}} {{NAME}}, Dr. JD, has been managing Mr. {{NAME}}'s complex cardiovascular condition, prescribing medications, providing dietary guidance, and coordinating with a multidisciplinary team to optimize his care. Mr. {{NAME}}'s condition remains stable, and he is actively engaged in cardiac rehabilitation and adhering to his medication regimen. The prognosis is guarded but hopeful with ongoing medical management and lifestyle modifications. Further interventions, such as revascularization procedures, may be considered in t

In [94]:
def upload(file_name):
  text = preview_text(file_name)
  return file_name, 'blah'

## Gradio App

In [35]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 9.7 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579

In [36]:
import gradio as gr

In [114]:
with gr.Blocks() as demo:
  with gr.Tab('Privatize Text'):
    text_input = gr.Textbox()

    with gr.Accordion("Advanced Options"):
      prompt_input = gr.Textbox(label = 'Prompt')
      generate_text_button = gr.Button("Generate Text")

    text_output = gr.Textbox(label = 'Privatized Text')
    text_button = gr.Button("Privatize")


  with gr.Tab('Privatize Image'):
    with gr.Row():
      image_input = gr.Image(source="upload", type="numpy", optional=False)
      image_output = gr.Image()
    image_button = gr.Button("Privatize")


  with gr.Tab('Privatize PDF'):
    with gr.Row():
      with gr.Row():
        aps_button = gr.Button("APS Demo")
        ca_button = gr.Button("Client Agreement Demo")
        mr_button = gr.Button("Medical Record Demo")
        pr_button = gr.Button("Payroll Demo")
      pdf_input = gr.File()
      preview_input = gr.Textbox(label = 'Preview', lines = 4)
    pdf_button = gr.Button("Privatize")
    pdf_summary = gr.Textbox(label = 'Document Summary', lines = 5)

  #aps_button.click(upload('APS_Demo_DSExpo.pdf'), aps_button, outputs = [pdf_input, preview_input])
  aps_button.click(lambda x: 'APS_Demo_DSExpo.pdf', ca_button, outputs = pdf_input)
  ca_button.click(lambda x: 'ClientAgreement_Demo_DSExpo.pdf', ca_button, outputs = pdf_input)
  mr_button.click(lambda x: 'MedicalRecord_Demo_DSExpo.pdf', mr_button, outputs = pdf_input)
  pr_button.click(lambda x: 'Payroll_Demo_DSExpo.pdf', pr_button, outputs = pdf_input)

  pdf_input.change(lambda x: preview_text(x.name), inputs = pdf_input, outputs = preview_input)

  generate_text_button.click(lambda x: x, inputs = prompt_input, outputs = text_input)

  text_button.click(privatize_text, inputs = text_input, outputs = text_output)
  image_button.click(privatize_img, inputs = image_input, outputs = image_output)
  pdf_button.click(lambda x: privatize_pdf_summary(x.name), inputs = pdf_input, outputs = pdf_summary)

<ipython-input-114-044cc2c3474e>:15: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  image_input = gr.Image(source="upload", type="numpy", optional=False)


In [112]:
demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>